In [1]:
#libraries import
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Decision to use BeautifulSoup for Toronto data

In [2]:
# webpage link, https://www.youtube.com/watch?v=ng2o98k983k
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(link).text
Toronto_data = BeautifulSoup(html, 'lxml')

In [3]:
# dataframe creation
column_names = ['Postalcode','Borough','Neighborhood']
Toronto = pd.DataFrame(columns = column_names)

Looking for the data

In [4]:
# postalcode, borough, neighborhood info finding in webpage
content = Toronto_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    Toronto = Toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean 
Toronto = Toronto[Toronto.Borough!='Not assigned']
Toronto = Toronto[Toronto.Borough!= 0]
Toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,Toronto.shape[0]):
    if Toronto.iloc[i][2] == 'Not assigned':
        Toronto.iloc[i][2] = Toronto.iloc[i][1]
        i = i+1
                                 
df = Toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head(10)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Data cleaning - drop all empty and Not assigned values. Not assigned values will be replaced by NaN. 

In [5]:
# data cleaning
df= df.dropna()
NA = 'Not assigned'
df = df[(df.Postalcode != NA ) & (df.Borough != NA) & (df.Neighborhood != NA)]

In [6]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
group_1 = df.groupby(['Postalcode', 'Borough'])
df_2 = group_1.apply(neighborhood_list).reset_index(name='Neighborhood')

In [8]:
print(df_2.shape)

(103, 3)


In [9]:
#Export dataset to .csv file, since it will be used on part 2
df_2.to_csv('Toronto_part1.csv')